In [1]:
import numpy as np
import pandas as pd
import os
from function import Slope, minSSE_recovery

s_c = '結算價'
top_path = './../../../'
Data_path = top_path + 'InterpData/'
expiry = 'NearbyMonth'

IV_type = 'callIV'
K_Range = [300, 500]
K_Range_file = '{}_{}.csv'.format(K_Range[0], K_Range[1])
K_Range_dir = '{}_{}'.format(K_Range[0], K_Range[1])
IV_path = '{}/{}/{}/{}'.format(Data_path, expiry, IV_type, K_Range_file)

Dir_tree = [top_path, 'ForecastData', expiry, IV_type, K_Range_dir]
current_path = Dir_tree[0]
for i in range(1, len(Dir_tree), 1):
    if Dir_tree[i] not in os.listdir(current_path):
        os.mkdir(current_path + Dir_tree[i])
    current_path = current_path + Dir_tree[i] + '/'

IV_data = pd.read_csv(IV_path, encoding='Big5', index_col=False)
IV_matrix = np.array(IV_data)

#----------------------------------------------------------------------------


OC = np.array(IV_data['期貨開盤價'] - IV_data['期貨收盤價'])
HL = np.array(IV_data['期貨最高價'] - IV_data['期貨最低價'])
expirty_days = np.array(IV_data['到期天數'])
#OC_HL_K_E = np.vstack((OC, HL, Kmin_reduce_F, Kmax_reduce_F, expirty_days)).T
OCHL = np.vstack((OC, HL)).T


IV_matrix = np.array(IV_data)
K_num = len(np.where(IV_matrix[0, 0] == IV_matrix)[0])
K = np.array(IV_data['履約價'])
K = np.reshape(K, (-1, K_num))
IV= np.array(IV_data['隱含波動率({})'.format(s_c)])
IV = np.reshape(IV, (-1, K_num))
K_IVslope = Slope(X=K, Y=IV, axis=1)
E = IV_matrix[range(0, len(IV_matrix), K_num), IV_data.columns.get_loc('到期天數')]

In [2]:
one_day_expiry_idx = np.arange(len(E))[np.equal(E, 1)]
if one_day_expiry_idx[-1] == len(E)-1:
    one_day_expiry_idx = one_day_expiry_idx[:-1]
most_days_expiry_idx = one_day_expiry_idx + 1
most_days_expiry_idx = np.hstack((0, most_days_expiry_idx))

most_days_expiry_idx0 = most_days_expiry_idx[:-1]
most_days_expiry_idx1 = most_days_expiry_idx[1:]
trade_days_in_month = most_days_expiry_idx1 - most_days_expiry_idx0  
most_days_expiry = E[most_days_expiry_idx]

contract_appear_days1 = [list(range(1, trade_day_in_month+1, 1)) \
                for trade_day_in_month in trade_days_in_month]
contract_appear_days1 = [contract_appear_day1 for subcontract_appear_day1 in contract_appear_days1\
                         for contract_appear_day1 in subcontract_appear_day1]
contract_appear_days1 = np.array(contract_appear_days1)
contract_appear_days2 = np.arange(1, len(E) - most_days_expiry_idx[-1] +1, 1)
contract_appear_days = np.hstack((contract_appear_days1, contract_appear_days2))
#contract_appear_days 為該契約(特定交易日期、到期日期，履約價不限)上市的交易日數，例如第一天上市到期天數35天，
#則值為1，第二天則到期日為34天值為2，一直到k(因為是交易日數，所以數字不一定)。然後又到下個月的契約，值又從1開
#始


from function import TimeSeriesData
seq_length1 = 7
seq_length2 = seq_length1 + 1
#seq_length2 = 16
min_contract_day = 2
magnification_slope = 10000


#變數有考慮到期日
#K_IVslope_E = np.hstack((K_IVslope*magnification_slope, np.reshape(E, (-1, 1))))

#Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, \
                                      #drop_out_columns=[len(K_IVslope_E[0])-1])

#變數沒考慮到期日
K_IVslope_E = K_IVslope * magnification_slope
Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, drop_out_columns=[])



Inputs_OCHLE  = OCHL[range(0, len(IV_matrix), K_num)]
Inputs_OCHLE  = np.array([Inputs_OCHLE[i:i+seq_length2] \
                              for i in range(len(Inputs_OCHLE)-seq_length2+1)])
Inputs_OCHLE  = Inputs_OCHLE.astype(float)
if seq_length1  > seq_length2-1:
    Inputs_OCHLE = Inputs_OCHLE[seq_length1-seq_length2+1:]
    contract_appear_days = contract_appear_days[seq_length1-seq_length2+1:]
    IV_matrix_forecast = IV_matrix[seq_length1*K_num:]
if seq_length1 <= seq_length2-1:
    Inputs_slope = Inputs_slope[seq_length2-1-seq_length1:]
    Ouputs = Ouputs[seq_length2-1-seq_length1:]
    contract_appear_days = contract_appear_days[seq_length2-1:]
    IV_matrix_forecast = IV_matrix[(seq_length2-1)*K_num:]


match_cond = np.greater_equal(contract_appear_days, min_contract_day)
Inputs_OCHLE = Inputs_OCHLE[match_cond]
Inputs_slope = Inputs_slope[match_cond]
contract_appear_days_filter = contract_appear_days[match_cond]
contract_appear_days_filter_Knum = contract_appear_days_filter.repeat(K_num)
match_cond_Knum = match_cond.repeat(K_num)
IV_matrix_forecast = IV_matrix_forecast[match_cond_Knum]
IV_matrix_forecast= np.hstack((IV_matrix_forecast, np.reshape(contract_appear_days_filter_Knum, (-1, 1))))
Ouputs = Ouputs[match_cond]



train_size = int(len(Inputs_slope)*0.8)
#val_size = int(train_size*0.2)
X_train = Inputs_slope[:train_size]
#X_train = Inputs_slope[:train_size-val_size]
#X_val = Inputs_slope[train_size - val_size:train_size]
OCHLE_train = Inputs_OCHLE[:train_size ]
#OCHLE_train = Inputs_OCHLE[:train_size - val_size]
#OCHLE_val = Inputs_OCHLE[train_size - val_size:train_size]
y_train = Ouputs[:train_size]
#y_train = Ouputs[:train_size-val_size]
#y_val = Ouputs[train_size - val_size:train_size]


X_test = Inputs_slope[train_size:]
y_test = Ouputs[train_size:]
IV_matrix_test = IV_matrix_forecast[K_num*train_size:]
OCHLE_test = Inputs_OCHLE[train_size:]


In [3]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, concatenate
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from skopt.acquisition import gaussian_ei

#進行調參的次數
n_calls = 30
#此函式可以幫我們設定一個模型的結構
def set_LSTM_LSTM(lstm1_structure, lstm2_structure, Megred_struture):
    lstm1_neurons_num = lstm1_structure['lstm_neurons_num']
    lstm1_activations = lstm1_structure['lstm_activations']
    lstm1_recurrent_activations = lstm1_structure['lstm_recurrent_activations']
    dense1_neurons_num = lstm1_structure['dense_neurons_num']
    dense1_activations = lstm1_structure['dense_activations']
    kernel_regularizer1= lstm1_structure['kernel_regularizer']

    lstm2_neurons_num = lstm2_structure['lstm_neurons_num']
    lstm2_activations = lstm2_structure['lstm_activations']
    lstm2_recurrent_activations = lstm2_structure['lstm_recurrent_activations']
    dense2_neurons_num = lstm2_structure['dense_neurons_num']
    dense2_activations = lstm2_structure['dense_activations']
    kernel_regularizer2= lstm2_structure['kernel_regularizer']
    
    Merged_neurons_num = Megred_struture['neurons_num']
    Megred_activations = Megred_struture['activations']
    Merged_kernel_regularizer = Megred_struture['kernel_regularizer']

    input_lstm1 = Input(shape=lstm1_structure['input_shape'])
    hidden_1 = input_lstm1

    for i in range(0, len(lstm1_neurons_num), 1):
        if i == len(lstm1_neurons_num) - 1:
            return_sequences=False
        else:
            return_sequences=True 
        hidden_1 = LSTM(lstm1_neurons_num[i], return_sequences=return_sequences,\
        activation=lstm1_activations[i], kernel_regularizer=kernel_regularizer1[i],
        recurrent_activation= lstm1_recurrent_activations[i])(hidden_1)
    output_1 = hidden_1
    for i in range(0, len(dense1_neurons_num), 1):
        output_1 = Dense(dense1_neurons_num[i], \
            kernel_regularizer=kernel_regularizer1[len(lstm1_neurons_num)-1+i], \
            activation=dense1_activations[i])(output_1)
        
    input_lstm2 = Input(shape=lstm2_structure['input_shape'])
    hidden_2 = input_lstm2

    for i in range(0, len(lstm2_neurons_num), 1):
        if i == len(lstm2_neurons_num) - 1:
            return_sequences=False
        else:
            return_sequences=True 
        hidden_2 = LSTM(lstm2_neurons_num[i], return_sequences=return_sequences,\
        activation=lstm2_activations[i], kernel_regularizer=kernel_regularizer2[i],
        recurrent_activation= lstm2_recurrent_activations[i])(hidden_2)
    output_2 = hidden_2
    for i in range(0, len(dense2_neurons_num), 2):
        output_2 = Dense(dense2_neurons_num[i], \
            kernel_regularizer=kernel_regularizer2[len(lstm2_neurons_num)-1+i], \
            activation=dense2_activations[i])(output_2)
        
    merged = concatenate([output_1, output_2], axis=-1)
    dense = merged
    for i in range(len(Merged_neurons_num)):
        dense = Dense(Merged_neurons_num[i], Megred_activations[i], Merged_kernel_regularizer[i])(dense)

    output = Dense(y_train.shape[1])(dense)
    model = Model(inputs=[input_lstm1, input_lstm2], outputs=output)
    return model

search_space = [
    Integer(100, 600, name='lstm1_neurons_num'),
    Integer(100, 600, name='lstm2_neurons_num'),
    Integer(1, 5, name='lstm1_layers_num'),
    Integer(1, 5, name='lstm2_layers_num'),
    Integer(100, 600, name='dense1_neurons_num'),
    Integer(100, 600, name='dense2_neurons_num'),
    Integer(100, 600, name='merged_neurons_num'),
    Integer(1, 3, name='merged_layers_num'),
    Real(1e-6, 1e-2, prior='log-uniform', name='lambda1'),
    Real(1e-6, 1e-2, prior='log-uniform', name='lambda2'),
    Real(1e-6, 1e-2, prior='log-uniform', name='lambda3')
]


def train_and_evaluate_net(lstm1_neurons_num, lstm2_neurons_num, lstm1_layers_num,\
                           lstm2_layers_num, dense1_neurons_num, dense2_neurons_num,
                           merged_neurons_num, merged_layers_num, lambda1, lambda2, lambda3):
    batch_size = 32
    epochs = 1000

    OCHL_lstm_neurons_num = [lstm1_neurons_num]*lstm1_layers_num 
    OCHLE_lstm_activations = ['tanh']*len(OCHL_lstm_neurons_num)
    OCHL_lstm_recurrent_activations = ['sigmoid']*len(OCHL_lstm_neurons_num)
    OCHL_dense_neurons_num = [dense1_neurons_num]
    OCHL_dense_activations = ['relu']
    OCHL_kernel_regularizer=[l2(lambda1)]*(len(OCHL_lstm_neurons_num) + len(OCHL_dense_neurons_num))

    Slope_lstm_neurons_num = [lstm2_neurons_num]*lstm2_layers_num
    Slope_lstm_activations = ['tanh']*len(Slope_lstm_neurons_num)
    Slope_lstm_recurrent_activations = ['sigmoid']*len(Slope_lstm_neurons_num)
    Slope_dense_neurons_num = [dense2_neurons_num]
    Slope_dense_activations = ['relu']
    Slope_kernel_regularizer=[l2(lambda2)]*(len(Slope_lstm_neurons_num) + len(Slope_dense_neurons_num))

    Merged_neurons_num = [merged_neurons_num] * merged_layers_num
    Megred_activations = ['relu']*len(Merged_neurons_num)
    Merged_kernel_regularizer=[l2(lambda3)]*(len(Merged_neurons_num))

    lstm_OCHL_structure = {
        'input_shape': OCHLE_train.shape[1:],
        'lstm_neurons_num': OCHL_lstm_neurons_num,
        'lstm_activations': OCHLE_lstm_activations,
        'lstm_recurrent_activations':OCHL_lstm_recurrent_activations,
        'dense_neurons_num':OCHL_dense_neurons_num,
        'dense_activations':OCHL_dense_activations,
        'kernel_regularizer':OCHL_kernel_regularizer
        }

    lstm_Slope_structure = {
        'input_shape': X_train.shape[1:],
        'lstm_neurons_num': Slope_lstm_neurons_num,
        'lstm_activations': Slope_lstm_activations,
        'lstm_recurrent_activations':Slope_lstm_recurrent_activations,
        'dense_neurons_num':Slope_dense_neurons_num,
        'dense_activations':Slope_dense_activations,
        'kernel_regularizer':Slope_kernel_regularizer
        }

    Merged_structure = {
        'neurons_num': Merged_neurons_num,
        'activations': Megred_activations,
        'kernel_regularizer': Merged_kernel_regularizer
        }
    alltrain = 1
    allMSE = np.zeros(alltrain) 
    for j in range(alltrain):
        #設定模型結構
        model = set_LSTM_LSTM(lstm_OCHL_structure, lstm_Slope_structure, Merged_structure)
        model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  

        # 訓練模型
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10, \
                                verbose=2, mode='min', restore_best_weights=True)
        hist_model = model.fit(
            [OCHLE_train, X_train], y_train, epochs=epochs, batch_size=batch_size, \
            validation_split=0.2, callbacks=[early_stopping]
        )

        #得到輸出
        y_pred = model.predict([OCHLE_test, X_test])
        y_pred = y_pred / magnification_slope
        columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
        column_index = [IV_data.columns.get_loc(col) for col in columns_names]

        SSE_everyday = np.zeros(((len(y_pred), K_num)))
    
        for i in range(len(y_pred)):
            K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
            iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
            y_pred_day = y_pred[i]

            ForecastData, SSE_everyday[i] = minSSE_recovery(y=iv_day, x=K_day, slope_yhat=y_pred_day)
        sse_everyday = SSE_everyday[:, 0]
        allMSE[j] = np.mean(sse_everyday)

    loss = np.min(allMSE)
    return loss

@use_named_args(search_space)
def objective(**params):
    lstm1_neurons_num = params['lstm1_neurons_num']
    lstm2_neurons_num = params['lstm2_neurons_num']
    lstm1_layers_num = params['lstm1_layers_num']
    lstm2_layers_num = params['lstm2_layers_num']
    dense1_neurons_num = params['dense1_neurons_num']
    dense2_neurons_num = params['dense2_neurons_num']
    merged_neurons_num = params['merged_neurons_num']
    merged_layers_num = params['merged_layers_num']
    lambda1 = params['lambda1']
    lambda2 = params['lambda2']
    lambda3 = params['lambda3']
    
    loss = train_and_evaluate_net(lstm1_neurons_num, lstm2_neurons_num, lstm1_layers_num,
                                  lstm2_layers_num, dense1_neurons_num, dense2_neurons_num,
                                  merged_neurons_num, merged_layers_num, lambda1, lambda2, lambda3)
    return loss


result = gp_minimize(objective, search_space, n_calls=n_calls, random_state=42, acq_func='EI')

# 输出最佳参数
print("最佳參數: {}".format(result.x))
print("最小損失: {}".format(result.fun))


best_params = result.x
lstm1_neurons_num = best_params[0]
lstm2_neurons_num = best_params[1]
lstm1_layers_num = best_params[2]
lstm2_layers_num = best_params[3]
dense1_neurons_num = best_params[4]
dense2_neurons_num = best_params[5]
merged_neurons_num = best_params[6]
merged_layers_num = best_params[7]
lambda1 = best_params[8]
lambda2 = best_params[9]
lambda3 = best_params[10]


Epoch 1/1000
56/56 [==============================] - 13s 74ms/step - loss: 4.1236 - mse: 3.8528 - val_loss: 1.1801 - val_mse: 0.9589
Epoch 2/1000
56/56 [==============================] - 2s 43ms/step - loss: 3.3003 - mse: 3.0884 - val_loss: 1.0081 - val_mse: 0.8063
Epoch 3/1000
56/56 [==============================] - 2s 44ms/step - loss: 3.0212 - mse: 2.8286 - val_loss: 0.8235 - val_mse: 0.6397
Epoch 4/1000
56/56 [==============================] - 3s 45ms/step - loss: 2.9189 - mse: 2.7414 - val_loss: 0.8590 - val_mse: 0.6888
Epoch 5/1000
56/56 [==============================] - 2s 44ms/step - loss: 2.9116 - mse: 2.7467 - val_loss: 0.8674 - val_mse: 0.7060
Epoch 6/1000
56/56 [==============================] - 2s 44ms/step - loss: 2.8030 - mse: 2.6455 - val_loss: 0.8032 - val_mse: 0.6503
Epoch 7/1000
56/56 [==============================] - 3s 45ms/step - loss: 2.7788 - mse: 2.6289 - val_loss: 0.7925 - val_mse: 0.6445
Epoch 8/1000
56/56 [==============================] - 2s 44ms/step -

In [4]:
import json
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
def convert_to_serializable(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    elif isinstance(obj, l2):
        return {"type": "L2", "l2": obj.l2}
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")
    


#決定這次的模型編號
model_idx = 1
#本次預測的變數為：slope(隱波與履約價之間的斜率)或iv(隱波的數值)
forecast_variable = 'slope'
#資料設定都相同的情況下，最多儲存多少種模型結果
max_model = 20

#設定參數
batch_size = 500
epochs = 1000


OCHL_lstm_neurons_num = [lstm1_neurons_num]*lstm1_layers_num 
OCHLE_lstm_activations = ['tanh']*len(OCHL_lstm_neurons_num)
OCHL_lstm_recurrent_activations = ['sigmoid']*len(OCHL_lstm_neurons_num)
OCHL_dense_neurons_num = [dense1_neurons_num]
OCHL_dense_activations = ['relu']
OCHL_kernel_regularizer=[l2(lambda1)]*(len(OCHL_lstm_neurons_num) + len(OCHL_dense_neurons_num))

Slope_lstm_neurons_num = [lstm2_neurons_num]*lstm2_layers_num
Slope_lstm_activations = ['tanh']*len(Slope_lstm_neurons_num)
Slope_lstm_recurrent_activations = ['sigmoid']*len(Slope_lstm_neurons_num)
Slope_dense_neurons_num = [dense2_neurons_num]
Slope_dense_activations = ['relu']
Slope_kernel_regularizer=[l2(lambda2)]*(len(Slope_lstm_neurons_num) + len(Slope_dense_neurons_num))

Merged_neurons_num = [merged_neurons_num] * merged_layers_num
Megred_activations = ['relu']*len(Merged_neurons_num)
Merged_kernel_regularizer=[l2(lambda3)]*(len(Merged_neurons_num))

lstm_OCHL_structure = {
    'input_shape': OCHLE_train.shape[1:],
    'lstm_neurons_num': OCHL_lstm_neurons_num,
    'lstm_activations': OCHLE_lstm_activations,
    'lstm_recurrent_activations':OCHL_lstm_recurrent_activations,
    'dense_neurons_num':OCHL_dense_neurons_num,
    'dense_activations':OCHL_dense_activations,
    'kernel_regularizer':OCHL_kernel_regularizer
}

lstm_Slope_structure = {
    'input_shape': X_train.shape[1:],
    'lstm_neurons_num': Slope_lstm_neurons_num,
    'lstm_activations': Slope_lstm_activations,
    'lstm_recurrent_activations':Slope_lstm_recurrent_activations,
    'dense_neurons_num':Slope_dense_neurons_num,
    'dense_activations':Slope_dense_activations,
    'kernel_regularizer':Slope_kernel_regularizer
}

Merged_structure = {
    'neurons_num': Merged_neurons_num,
    'activations': Megred_activations,
    'kernel_regularizer': Merged_kernel_regularizer
}

structure_list = [lstm_OCHL_structure, lstm_Slope_structure, Merged_structure]
structure_name_list = ['lstm_OCHL_structure', 'lstm_Slope_structure', 'Merged_structure']
General_structure = {
    structure_name_list[0] : structure_list[0],
    structure_name_list[1]: structure_list[1],
    structure_name_list[2] : structure_list[2]
}


def save_model_txt(file_path, structure_list, structure_name_list, mse, mse_adj):
    with open('{}'.format(file_path), 'w', encoding='utf-8') as txt_file:
        for i in range(len(structure_list)):
            txt_file.write(structure_name_list[i] + ':\n')
            for key, value in structure_list[i].items():
                txt_file.write(f"{key}: {value}\n")
            txt_file.write('-----------------\n')
        txt_file.write('MSE:{}'.format(mse))
        txt_file.write('MSE_adj:{}'.format(mse_adj))


alltrain = 5
allMSE = np.zeros(alltrain)
allMSE_adj = np.zeros(alltrain)  
allForecastIV = np.zeros((K_num*len(OCHLE_test), alltrain))
allSSE_everyday = np.zeros((K_num*len(OCHLE_test), alltrain))
allSSE_everyday_adj = np.zeros((K_num*len(OCHLE_test), alltrain))
for j in range(alltrain):
    #設定模型結構
    model = set_LSTM_LSTM(lstm_OCHL_structure, lstm_Slope_structure, Merged_structure)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  

    # 訓練模型
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10, \
                               verbose=2, mode='min', restore_best_weights=True)
    hist_model = model.fit(
        [OCHLE_train, X_train], y_train, epochs=epochs, batch_size=batch_size, \
        validation_split=0.2, callbacks=[early_stopping]
    )




    #得到輸出
    y_pred = model.predict([OCHLE_test, X_test])
    y_pred = y_pred / magnification_slope
    columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
    column_index = [IV_data.columns.get_loc(col) for col in columns_names]
    ForecastIV = np.zeros(((len(y_pred), K_num)))
    SSE_everyday = np.zeros(((len(y_pred), K_num)))
    SSE_everyday_adj = np.zeros(((len(y_pred), K_num)))

    model_name = 'model{}'.format(model_idx)
    #model_file = 'model{}.h5'.format(model_idx)
    model_png = 'model{}.png'.format(model_idx)
    Forecast_name = 'model{}.csv'.format(model_idx)
    model_txt = 'model{}.txt'.format(model_idx)
    model_json = 'model{}.json'.format(model_idx)
    for i in range(len(y_pred)):
        K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
        iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
        y_pred_day = y_pred[i]
        iv_mean = iv_day / np.mean(iv_day)
        ForecastIV[i], sse_day = minSSE_recovery(y=iv_day, x=K_day, slope_yhat=y_pred_day)
        SSE_everyday[i] = sse_day
        SSE_everyday_adj[i] = sse_day / iv_mean

    ForecastIV = np.reshape(ForecastIV,(-1,1))
    sse_everyday = SSE_everyday[:, 0]
    sse_everyday_adj = SSE_everyday_adj[:, 0]
    SSE_everyday = np.reshape(SSE_everyday ,(-1,1))
    SSE_everyday_adj = np.reshape(SSE_everyday_adj ,(-1,1))
    allMSE[j] = np.mean(sse_everyday)
    allMSE_adj[j] = np.mean(sse_everyday_adj)
    allForecastIV[:,j:j+1] = ForecastIV
    allSSE_everyday[:, j:j+1] = SSE_everyday
    allSSE_everyday_adj[:, j:j+1] = SSE_everyday_adj

min_MSE_idx = np.argsort(allMSE)[0]
ForecastIV = allForecastIV[:, min_MSE_idx:min_MSE_idx+1]
SSE_everyday = allSSE_everyday[:, min_MSE_idx:min_MSE_idx+1]
SSE_everyday_adj = allSSE_everyday_adj[:, min_MSE_idx:min_MSE_idx+1]
min_MSE = allMSE[min_MSE_idx]
min_MSE_adj = allMSE_adj[min_MSE_idx]

Forecast_matrix  = np.hstack((IV_matrix_test, ForecastIV, SSE_everyday, SSE_everyday_adj))
column = np.hstack((IV_data.columns.to_numpy(), \
                        np.array(['上市天數(交易日)','預測隱含波動率({})'.format(s_c), 'loss', '調整後loss'])))
    
   
Forecast_Data = pd.DataFrame(data=Forecast_matrix, columns=column)


loss_columns_names = ['交易日期', '到期天數', '上市天數(交易日)', 'loss']
loss_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_columns_names]
loss_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_columns_index]
loss_Data = loss_Data.reset_index().iloc[:,1:]
loss_Data = loss_Data.rename(columns={'loss': model_name})
MSE_data = pd.DataFrame(columns=loss_Data.columns, data=[['MSE', 'MSE', 'MSE', min_MSE]])
loss_Data = pd.concat([loss_Data, MSE_data], axis=0)
loss_Data = loss_Data.reset_index().iloc[:,1:]
print(min_MSE)




model_path = top_path
model_type = 'LSTM-LSTM'

model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), model_type,\
                  'seq{}_seq{}_min{}'.format(seq_length1, seq_length2, min_contract_day), forecast_variable]
for model_dir in model_Dir_tree:
    if model_dir not in os.listdir(model_path):
        os.mkdir(model_path + model_dir)
    model_path = model_path + model_dir + '/'

loss_adj_columns_names = ['交易日期', '到期天數', '調整後loss']
loss_adj_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_adj_columns_names]
loss_adj_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_adj_columns_index]
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]
loss_adj_Data = loss_adj_Data.rename(columns={'調整後loss': model_name})
MSE_adj_data = pd.DataFrame(columns=loss_adj_Data.columns, data=[['MSE', 'MSE', min_MSE_adj]])
loss_adj_Data = pd.concat([loss_adj_Data, MSE_adj_data], axis=0)
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]



if 'Modelloss.csv' in os.listdir(model_path):
    Modelloss = pd.read_csv(model_path + 'Modelloss.csv', index_col=False, encoding='Big5')
    Modelloss_adj = pd.read_csv(model_path + 'Modelloss_adj.csv', index_col=False, encoding='Big5')
    if model_name in Modelloss.columns:
        if Modelloss[model_name][len(Modelloss)-1] > min_MSE:
            Modelloss[model_name] = loss_Data[model_name]
            Modelloss_adj[model_name] = loss_adj_Data[model_name]
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            #model.save(model_path + model_file)
            with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
            save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    elif len(Modelloss.columns) >= max_model+2 :
        all_MSE = np.array(Modelloss.iloc[-1, 2:])
        max_MSE = np.max(all_MSE)
        if max_MSE > min_MSE:
            max_MSE_idx = (np.arange(len(all_MSE))[np.equal(all_MSE, max_MSE)])[0] +2
            Modelloss.iloc[:,max_MSE_idx] = loss_Data[model_name]
            Modelloss_adj.iloc[:,max_MSE_idx] = loss_adj_Data[model_name]
            model_name = Modelloss.columns[max_MSE_idx]
            Forecast_name = model_name + '.csv'
            #model_file = model_name + '.h5'
            model_png = model_name + '.png'
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            #model.save(model_path + model_file)
            with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
            save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    else:
        Modelloss = pd.concat([Modelloss, loss_Data[model_name]], axis=1)
        Modelloss = Modelloss.reset_index().iloc[:,1:] 
        Modelloss_adj = pd.concat([Modelloss_adj, loss_Data[model_name]], axis=1)
        Modelloss_adj = Modelloss_adj.reset_index().iloc[:,1:] 
        Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
        #model.save(model_path + model_file)
        with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
        save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
        plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
        print('此模型已經被儲存為{}'.format(model_name))
else:
    Modelloss = loss_Data
    Modelloss_adj = loss_adj_Data
    Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
    #model.save(model_path + model_file)
    with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
    save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
    plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)

    
Modelloss.to_csv(model_path + 'Modelloss.csv', index=False, encoding='Big5')
Modelloss_adj.to_csv(model_path + 'Modelloss_adj.csv', index=False, encoding='Big5')

Epoch 1/1000
4/4 [==============================] - 8s 544ms/step - loss: 5.4268 - mse: 4.7774 - val_loss: 1.8976 - val_mse: 1.3001
Epoch 2/1000
4/4 [==============================] - 0s 76ms/step - loss: 4.4263 - mse: 3.8525 - val_loss: 1.4466 - val_mse: 0.9194
Epoch 3/1000
4/4 [==============================] - 0s 79ms/step - loss: 3.6822 - mse: 3.1767 - val_loss: 1.2530 - val_mse: 0.7902
Epoch 4/1000
4/4 [==============================] - 0s 87ms/step - loss: 3.3015 - mse: 2.8585 - val_loss: 1.1375 - val_mse: 0.7332
Epoch 5/1000
4/4 [==============================] - 0s 87ms/step - loss: 3.1859 - mse: 2.7995 - val_loss: 1.0357 - val_mse: 0.6840
Epoch 6/1000
4/4 [==============================] - 0s 109ms/step - loss: 3.0455 - mse: 2.7096 - val_loss: 0.9574 - val_mse: 0.6524
Epoch 7/1000
4/4 [==============================] - 0s 81ms/step - loss: 2.9245 - mse: 2.6335 - val_loss: 0.8792 - val_mse: 0.6153
Epoch 8/1000
4/4 [==============================] - 0s 89ms/step - loss: 2.8364 -